In [1]:
import nltk
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

#===============
#line1="Dear Sir or Madame,  we already booked our room from Juli 29th to July 31st.  Hotel confirmation #138303907222  Two questions:  - Is there a (free) shuttle service from and to the airport. Or what is the best way to travel? - is breakfast included?? Best regards,  M. Zierer"
line1="where is it"
line2 = "Front desk told us that we will have a military discount when we arrived in the arrival they would not give it to us, even though we showed a valid military ID card. They carged us 384$, and when we booked they said the rate would be 224$ .... We won't be coming back!!! Bad business!!!"
#===============

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

#===============
print(nltk.classify.accuracy(classifier, test_set))
print(classifier.classify(dialogue_act_features(line1)))
print(classifier.classify(dialogue_act_features(line2)))
#===============

#def analyze_comments(comments):
#	return classifier.classify(dialogue_act_features(comments))

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
0.668
whQuestion
Clarify


In [12]:
def analyze_sentiment(input):
	import nltk
	import pandas as pd
	from nltk.sentiment.vader import SentimentIntensityAnalyzer
	from nltk.sentiment.util import extract_unigram_feats, mark_negation

	sid = SentimentIntensityAnalyzer()
	output_sentiment=""
	check_false_positive=mark_negation(input.split())
	ss = sid.polarity_scores(input)	
	if ss["compound"] == 0.0:
		output_sentiment="neutral"
	elif ss["compound"] > 0.0:		
		output_sentiment="positive"
		if "_NEG" in str(check_false_positive):			
			output_sentiment="negative"
	else:		
		output_sentiment="negative"

	return output_sentiment

def tokenize_analyze(input):
	from nltk.tokenize import sent_tokenize,word_tokenize, WordPunctTokenizer
	
	divide_input=sent_tokenize(input)
	list_length = len(divide_input)
	positive_ctr=0
	negative_ctr=0
	neutral_ctr=0

	for ctr in range(list_length):
		if analyze_sentiment(divide_input[ctr])=="positive":
			positive_ctr=positive_ctr+1
		elif  analyze_sentiment(divide_input[ctr])=="negative":		
			negative_ctr=negative_ctr+1		
		else:
			neutral_ctr=neutral_ctr

	if positive_ctr > negative_ctr:
		var_message='positive'
		if negative_ctr >= 1:
			var_message='negative'
	elif positive_ctr < negative_ctr:
		var_message='negative'
	else:
		var_message='neutral'

	return(var_message)

Sentence = ' I received a message that you did not have my CC on reservation SWYZS8MUE.  I did include it but since your system lost it, need to know the most secure way to get it to you without calling from the United States? My contact # is +1 310 795 9319 if you can call me that way.  I am on the West Coast of the United States.'
Sentence = Sentence.replace('dears','').replace('dear','').replace('hi','').replace('hello','')

print(Sentence)
print (tokenize_analyze(Sentence))
print (analyze_sentiment(Sentence))


 I received a message that you did not have my CC on reservation SWYZS8MUE.  I did include it but since your system lost it, need to know the most secure way to get it to you without calling from the United States? My contact # is +1 310 795 9319 if you can call me that way.  I am on the West Coast of the United States.
positive
negative
